# Teil 6 Demo 2: Geovisualisierung

*Hinweis:* Dieser Teil der Demo funktioniert leider nicht unter Google Colab, da Leaflet dort nicht dargestellt wird. Sie kann stattdessen auf einem lokal installierten Jupyter Notebook ausgeführt werden.

## Bibliotheken & Konfiguration

In [ ]:
# Mit der folgenden Zeile können in Google Colab bei Bedarf Systembibliotheken installiert werden, die für das "sf"-Package notwendig sind
#system('sudo apt-get install -y libudunits2-dev libgdal-dev libgeos-dev libproj-dev', intern=TRUE)

packages <- c("readr", "dplyr", "tidyr", "leaflet", "tidygeocoder", "sf", "ggmap")
install.packages(setdiff(packages, rownames(installed.packages())))
lapply(packages, require, character.only = TRUE)

base_url <- "https://raw.githubusercontent.com/ganslats/TMF-School-2021-Block-7/master/Rohdaten/covid/"

## Basiskarte mit Leaflet anzeigen

Hierzu wird der Mittelpunkt der Karte und die Zoomstufe festgelegt und ein Hintergrund ("Tiles") hinzugefügt. Defaultmäßig werden die Tiles von OpenStreetMap genutzt.

In [ ]:
leafletmap <- leaflet() %>%
    setView(10.217850, 51.151509, zoom=5) %>%
    addTiles()
leafletmap

### Hintergrund ändern

Es gibt eine Vielzahl von Anbietern & Varianten der Hintergrundkarten (Tiles)
* hier gibt es eine Übersicht aller verfügbaren Anbieter & Varianten: http://leaflet-extras.github.io/leaflet-providers/preview/index.html
* mit der Funktion "names(providers)" des Leaflet-Package können alle verfügbaren Anbieter/Varianten als Liste angezeigt werden

Um die Hintergrundkarte vorzugeben, wird *addTiles()* ersetzt durch *addProviderTiles(providers$ANBIETER.VARIANTE)*:

In [ ]:
leafletmap <- leaflet() %>%
    setView(13.3897338, 52.518763, zoom=15) %>%
    addProviderTiles(providers$Stamen.Watercolor)
leafletmap

## Marker auf die Karte setzen

### Einzelnen Marker platzieren

Mit der Funktion *addMarkers(LÄNGE, BREITE, label="Bezeichner", popup="Popuptext")* kann ein einzelner Marker auf der Karte platziert werden:

#### Einzelne Adresse geocodieren

Mit dem R-Package "tidygeocoder" kann eine Geocodierung über den freien Nominatim-Service von OpenStreetMap durchgeführt werden.

Zu beachten:
* der öffentliche Nominatim-Server ist nicht für die Geocodierung großer Mengen von Adressen vorgesehen; Usage Policy beachten (https://operations.osmfoundation.org/policies/nominatim/)
* unbedingt Datenschutzaspekte beachten: personenbeziehbare Adressen niemals über fremde Server geocodieren!
* Nominatim kann relativ leicht (z.B. Docker-basiert) lokal installiert werden. Zu beachten: die Kartendaten sind umfangreich und führen zu sehr langen Ladezeiten beim Aufsetzen des Servers (daher kleinstmögliche geografische Region auswählen, z.B. nur Deutschland)

In [ ]:
# Geocodierung über den Nominatim-Service von OpenStreetMap (method="osm") aufrufen & Ergebnis anzeigen
geo(address = "Charlottenstr 42, Berlin, Deutschland", method = "osm")

# Ergebnis manuell als Marker in Karte platzieren
leafletmap <- leaflet() %>%
    setView(13.39033, 52.5186, zoom=15) %>%
    addTiles() %>%
    addMarkers(13.39033, 52.5186, label="TMF", popup="TMF")
leafletmap

### Mehrere Marker aus Datensatz platzieren

Marker werden üblicherweise gebündelt über Tibbles auf der Karte platziert.
Hierzu muss in der *AddMarkers*-Funktion eine Datenquelle angegeben werden, und die einzelnen Attribute werden aus Spalten des Tibbles befüllt, statt sie explizit anzugeben:

In [ ]:
# Accesspoints des öffentlichen WLANs aus dem Mannheimer Open Data-Portal einlesen
mannheim.wifi.raw <- read_csv2("https://mannheim.opendatasoft.com/explore/dataset/standorte-wlan-mit-geocode/download/?format=csv&timezone=Europe/Berlin&lang=de&use_labels_for_header=true&csv_separator=%3B",
                               col_types = cols(Strasse = col_character(), `Haus Nr.` = col_number(), Postleitzahl = col_double(), Ort = col_character(), Land = col_character(), Bundesland = col_character(), Adresse = col_character(), Geocode = col_character(), Einrichtung = col_character(), Anmerkungen = col_character()))

# Länge/Breite aus kombiniertem Feld splitten
mannheim.wifi.raw <- mannheim.wifi.raw %>% separate(col = Geocode, into = c("lat", "lon"), sep=",", convert = TRUE)

leafletmap <- leaflet(mannheim.wifi.raw) %>%
    setView(8.4772773, 49.4850955, zoom=13) %>%
    addTiles() %>%
    addMarkers(data = mannheim.wifi.raw, 
               ~lon, 
               ~lat,
               label = ~Adresse)
leafletmap

### Marker clustern, um die Übersichtlichkeit zu erhöhen

Bei vielen Markern kann es gerade in herausgezoomter Ansicht sehr unübersichtlich werden. Durch Clustering werden nahe beieinanderliegende Marker zusammengefasst, wobei die Farbe des Clusters von der Zahl der enthaltenen Marker abhängt (heatmap-artig). Beim Heranzoomen werden die Cluster verkleinert oder wieder zu Einzelmarkern aufgelöst.

Zum Clustern muss innerhalb der *AddMarkers*-Funktion die Option *clusterOptions = markerClusterOptions()* ergänzt werden:

In [ ]:
leafletmap <- leaflet(mannheim.wifi.raw) %>%
    setView(8.4772773, 49.4850955, zoom=13) %>%
    addTiles() %>%
    addMarkers(data = mannheim.wifi.raw, 
               ~lon, 
               ~lat,
               label = ~Adresse,
               clusterOptions = markerClusterOptions())
leafletmap

## Regionen mit Choroplethen darstellen

### Shapes für die administrativen Regionen in Deutschland laden

Shape liegen in verschiedenen Formaten vor, die alle von Leaflet verarbeitet werden können:
* Shapefiles (.shp): älteres Format, besteht aus ZIPfile mit mehreren Dateien für die Polygone und begleitende Informationen
* Geopackages: einfaches Datenbankformat (auf Basis von SQLite), das sowohl die Polyogone als auch begleitende Informationen in einer Datei enthält
* GeoJSON: strukturiertes Format (JavaScript Object Notation), das Polygone & begleitende Informationen in einer Datei enthält

Die Geodaten für die Übung werden von https://gadm.org/ als Geopackages bezogen, wo sowohl Shapefiles als auch Geopackages heruntergeladen werden können. Die Lizenz erlaubt keine Redistribution, so dass die Datei im Kurs direkt von dort geladen wird. Hierzu die ersten beiden Zeilen der folgenden Zelle aktivieren, für spätere Läufe können sie wieder auskommentiert werden, da die Datei dann bereits lokal vorliegt.

In [ ]:
#download.file("https://biogeo.ucdavis.edu/data/gadm3.6/gpkg/gadm36_DEU_gpkg.zip", "gadm36_DEU_gpkg.zip")
#unzip("gadm36_DEU_gpkg.zip", "gadm36_DEU.gpkg", junkpaths = TRUE)
shapes.landkreise <- st_read("gadm36_DEU.gpkg", layer="gadm36_DEU_2")

# Polygone ausgeben (ohne Hintergrundkarte)
leaflet(data = shapes.landkreise) %>% 
    addPolygons(color = "black", 
                weight = 0.5, 
                opacity = 1, 
                label = ~NAME_2)

### Rohdaten für die Visualisierung: Covid19-Inzidenzen (pro 100.000 Bevölkerung) über alle Land-/Stadtkreise in Deutschland einlesen

Quellen: 
* COVID-19 Datenhub des RKI beim Bundesamt für Kartographie und Geodäsie (https://npgeo-corona-npgeo-de.hub.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0). Die Daten sind die „Fallzahlen in Deutschland“ des Robert Koch-Institut (RKI) und stehen unter der Open Data Datenlizenz Deutschland – Namensnennung – Version 2.0 zur Verfügung. (https://www.govdata.de/dl-de/by-2-0) Quellenvermerk: Robert Koch-Institut (RKI), dl-de/by-2-0
* Amtliches Gemeindeverzeichnis des Statistischen Bundesamts (https://www.destatis.de/DE/Themen/Laender-Regionen/Regionales/Gemeindeverzeichnis/Administrativ/04-kreise.html)

Aufbereitung für den Kurs: Auswahl relevanter Spalten, Aggregation von Teilmeldungen pro Tag auf Kreise, manuelles Mapping der Kreisbezeichner zwischen den Teildatensätzen, Berechnung der Fallzahl pro 100.000 Bevölkerung

In [ ]:
# Inzidenzdaten für die Kreise in Gesamtdeutschland laden
germany.incidence.raw <- read_csv2(paste(base_url, "covid_germany_incidence.csv", sep=""),
                                   col_types = cols(kreis_id = col_character(), kreis_name = col_character(), fallzahl = col_double(), fallzahl_pro_100k = col_double()))
head(germany.incidence.raw)

# Inzidenzen mit den Shapes der Kreise zusammenführen & Label für die Anzeige ergänzen
germany.incidence.map <- shapes.landkreise %>% left_join(germany.incidence.raw, by = c("CC_2" = "kreis_id"))
germany.incidence.map$label <- paste0(germany.incidence.map$kreis_name, ": ", sprintf("%0.2f", germany.incidence.map$fallzahl_pro_100k), "/100.000 (", germany.incidence.map$fallzahl, " Fälle)")


## Geovisualisierung der Fallzahl/100.000 mit Leaflet

Hierzu wird zunächst mit der funktion *colorBin()* eine Farbpalette generiert und mit *addPolygons()* ein Choropletzen-Layer mit der Fallzahl und der generierten Palette dargestellt. Mit *addLegend()* wird eine Legende ergänzt.

In [ ]:
# Palette für die Darstellung der Inzedenzen berechnen
pal <- colorBin(
  palette = "Spectral",
  domain = germany.incidence.map$fallzahl_pro_100k,
  reverse = TRUE
)

# Karte ausgeben
leaflet(data = germany.incidence.map) %>% 
    addTiles() %>%
    addPolygons(color = "black", 
                weight = 0.5, 
                opacity = 1, 
                fillColor = ~pal(fallzahl_pro_100k), 
                fillOpacity = 0.5,
                label = ~label) %>%
    addLegend(pal = pal, values = ~fallzahl_pro_100k, title = "Fallzahl/100.000",
              position = "bottomright")

# Geovisualisierung mit ggmap

Im folgenden Teil der Demo werden die gleichen Schritte mit ggmap als statische Karten wiederholt. Dieser Teil funktioniert auch auf Google Colab.

## Basiskarte mit ggmap darstellen

Basiskarten können bei ggmap mit den *get_ANBIETERmap()*-Funktionen geladen werden. Es gibt allerdings mehrere Einschränkungen:
* die Funktion zum Laden von OpenStreetMap-Hintergrundkarten ist seit längerem deaktiviert, da sie auf ein dafür seitens OSM nicht vorgesehenes API zugreift und dort gesperrt wurde; es ist nicht erkennbar, dass an einer Lösung gearbeitet wird
* die Funktion zum Laden von GoogleMaps-Basiskarten erfordert die vorherige Anlage eines API-Keys bei Google; es gibt ein Rate-Limit und Maximalzahlen von Karten-(Tiles), die kostenfrei abgerufen werden können

In [ ]:
#boundingbox <- c(-0.14185925, 51.5108615, -0.13114775, 51.5168285)
boundingbox <- c(4.3451573, 47.2942196, 15.5267352, 54.776929)
map.germany <- get_stamenmap(bbox = boundingbox, maptype="terrain", crop=TRUE, zoom = 7)
options(repr.plot.width = 5, repr.plot.height = 5)
ggmap(map.germany)

## Manuellen Marker auf der Karte eintragen

Marker und Texte können mit den normalen geoms aus ggplot2 ergänzt werden. Koordinaten können dabei sowohl einzeln explizit als auch über Spalten eines Tibble angegeben werden.

In [ ]:
ggmap(map.germany) +
    geom_point(aes(x=13.39033, y=52.5186)) +
    geom_text(aes(x=13.39033, y=52.786, label="TMF"))

## Marker aus Datensatz einfügen

In [ ]:
# Boundingbox aus Datensatz generieren
boundingbox <- make_bbox(data = mannheim.wifi.raw, lon = lon, lat = lat, f = 0.05)
map.mannheim <- get_stamenmap(bbox = boundingbox, maptype="watercolor", crop=TRUE, zoom = 13)
options(repr.plot.width = 5, repr.plot.height = 5)
ggmap(map.mannheim) +
    geom_point(data = mannheim.wifi.raw, aes(x=lon, y=lat))

## Regionen mit Choroplethen in ggmap darstellen

In [ ]:
ggplot() +
    geom_sf(data = germany.incidence.map, aes(fill=fallzahl_pro_100k), size=0.001) +
    scale_fill_distiller(palette="Spectral", name="Fallzahl pro 100.000", na.value='#bbbbbb')
